# Cell Neighbors Analysis
There are two components of this notebook: **neighborhood diversity** and **cell distance analysis**. Each is independent of the other, so you can choose only to run what you're interested in; however, you must run the four cells below to set up the notebook paths and generate the distance matrices for your data.

In [ ]:
import os
import pandas as pd 
from ark.utils import example_dataset
import ark.settings as settings
from ark.analysis import spatial_analysis_utils
from ark.analysis.neighborhood_analysis import create_neighborhood_matrix
from ark.analysis.cell_neighborhood_stats import generate_neighborhood_diversity_analysis, generate_cell_distance_analysis

## Path Setup
* `base_dir`: the path to the main folder containing all of your imaging data. This directory will also store all of the directories/files created during analysis
* `spatial_analysis_dir`: the path to the directory containing the spatial analysis output
* `cell_table_path`: the path to the cell table that contains columns for fov, cell label, and cell phenotype (generated from `3_Pixie_Cluster_Cells.ipynb`)
* `dist_mat_dir`: the path to directory containing the distance matrices for your data
* `neighbors_mat_dir`: the path to directory containing the neighborhood matrix for your data
* `output_dir`: path for a new directory that will be created for the output below

In [ ]:
base_dir = '../data/example_dataset'

If you would like to test the features in Ark with an example dataset, run the cell below. It will download a dataset consisting of 11 FOVs with 22 channels. You may find more information about the example dataset in the [README](../README.md#example-dataset).

If you are using your own data, skip the cell below.

* `overwrite_existing`: If set to `False`, it will not overwrite existing data in the `data/example_dataset`. Recommended setting to `False` if you are running Notebooks 1,2,3 and 4 in succession. Set to `True` if you are just running Notebook 4.

In [ ]:
example_dataset.get_example_dataset(dataset="post_clustering", save_dir=base_dir, overwrite_existing=False)

In [ ]:
spatial_analysis_dir = os.path.join(base_dir, "spatial_analysis")
segmentation_dir = os.path.join(base_dir, "segmentation/deepcell_output")
cell_table_path = os.path.join(base_dir, "segmentation/cell_table/cell_table_size_normalized_cell_labels.csv")
dist_mat_dir = os.path.join(spatial_analysis_dir, "dist_mats")

neighbors_mat_dir = os.path.join(spatial_analysis_dir, "neighborhood_mats")
output_dir = os.path.join(spatial_analysis_dir, "cell_neighbor_analysis")

In [ ]:
# generate distance matrices if needed
if not os.path.exists(dist_mat_dir):
    os.makedirs(dist_mat_dir)
    spatial_analysis_utils.calc_dist_matrix(segmentation_dir, dist_mat_dir)

# create neighbors matrix and output directories
for directory in [neighbors_mat_dir, output_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

## Cell Neighborhood Diversity
This part of the notebook can be used to determine the homogeneity/diversity of the neighbors surrounding each of the cells in our images.

### 1. Neighborhood Matrices
You must specify which neighbors matrix should be used based on the pixel radius and cell type column.
- `pixel_radius`: radius used to define the neighbors of each cell
- `cell_type_col`: column in your cell table containing the cell phenotypes

**If you have not previously a generated neighbors matrix with the `pixel_radius` and `cell_type_col` specified, it will be created below.**

In [ ]:
pixel_radius = 50
cell_type_col = settings.CELL_TYPE

In [ ]:
freqs_path = os.path.join(neighbors_mat_dir, f"neighborhood_freqs-{cell_type_col}_radius{pixel_radius}.csv")
    
# Check for existing neighbors matrix and if not, create a new one
if not os.path.exists(freqs_path):
    print(f"Generating neighbors matrix for {cell_type_col}.")
    
    all_data = pd.read_csv(cell_table_path)
    _, neighbor_freqs = create_neighborhood_matrix(
        all_data, dist_mat_dir, distlim=pixel_radius, cell_type_col=cell_type_col)
    
    # Save neighbors frequency matrix
    neighbor_freqs.to_csv(freqs_path, index=False)

### 2. Compute Shannon Diversity
The code below will calculate the Shannon Diversity Index for each cell input. **The resulting file will be saved to the save_path (`neighborhood_diversity_radius{pixel_radius}.csv`) in the new cell_neighbor_analysis subdirectory.**

In [ ]:
cell_type_columns = [cell_type_col] 
save_path = os.path.join(output_dir, f'neighborhood_diversity_{cell_type_col}_radius{pixel_radius}.csv')

diversity_data = generate_neighborhood_diversity_analysis(neighbors_mat_dir, pixel_radius, cell_type_columns)
diversity_data.to_csv(save_path, index=False)

## Cell Distances
This part of the notebook can be used to analyze the proximty/distance between cell phenotypes in samples. 

### 1. Compute Average Distances
For every cell in an image, the code below will compute the average distance of the *k* closest cells of each phenotype. You must specify *k* and the cell phenotype column name below.

- `k`: how many nearest cells of a specific phenotype to average the distance over
- `cell_type_col`: column in your cell table containing the cell phenotypes

**The resulting file will be saved to the save_path (`{cell_type_col}_avg_dists-nearest_{k}.csv`) in the new cell_neighbor_analysis subdirectory.**

In [ ]:
k = 5
cell_type_col = settings.CELL_TYPE

In [ ]:
all_data = pd.read_csv(cell_table_path)
    
save_path = os.path.join(output_dir, f"{cell_type_col}_avg_dists-nearest_{k}.csv")
distance_data = generate_cell_distance_analysis(all_data, dist_mat_dir, save_path, k, cell_type_col)